In [2]:
import mne
import mne.io
import mne_bids
import numpy as np
import plotly.express as px
import q1k_sync_tools as qst

In [3]:
#subject_id = '0008'
#task = 'ssvep'
bids_root = '.'

In [ ]:
#read EEG mff data
eeg_raw = mne.io.read_raw_egi('sourcedata/s06/eeg/s06/s06_plr_20230313_115649.mff/',preload=True)
eeg_raw_df = eeg_raw.to_data_frame()

#get the EEG events from the stim stim channel
#eeg_events = mne.find_events(eeg_raw, stim_channel='STI 014', shortest_event = 1)
eeg_events = mne.find_events(eeg_raw, shortest_event = 1)

#create the event dictionary..
eeg_event_dict = qst.get_event_dict(eeg_raw,eeg_events)

In [ ]:
#handle task specific EEG event interpretation..
eeg_events, eeg_stims, eeg_iti, eeg_event_dict = qst.eeg_event_test(eeg_events,eeg_event_dict,task_name='plr')

In [ ]:
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.show()

In [ ]:
#read the asc eye tracking data and convert it to a dataframe
et_raw = mne.io.read_raw_eyelink ('sourcedata/s06/eyetracking/s06plr.asc')
et_raw_df = et_raw.to_data_frame()

#get the events from the annotation structure
et_annot_events, et_annot_event_dict = mne.events_from_annotations(et_raw)

In [15]:
#handle task specific ET event interpretation..
et_event_raw_df, et_events, et_stims, et_iti = qst.et_event_test(et_raw_df,task_name='plr')

Number of eye-tracking stimulus onset DIN events: 32


In [ ]:
fig=px.scatter(x=et_events['index'],y=et_events['DIN_diff'])
fig.show()

In [ ]:
qst.show_sync_offsets(eeg_stims,et_stims)

In [ ]:
eeg_et_raw = qst.eeg_et_combine(eeg_raw, et_raw, eeg_stims, et_stims)

In [18]:
eeg_et_raw_df = eeg_et_raw.to_data_frame()

In [ ]:
fig=px.line(x=eeg_et_raw_df['time'],y=eeg_et_raw_df['pupil_right'])
fig.show()

In [ ]:
scalings=dict(eyegaze=1e2)
eeg_et_raw.plot(events=eeg_events,duration=4,scalings=scalings)
#eeg_et_raw.plot(scalings=scalings)

In [ ]:
fig=px.scatter(x=et_events['index'],y=et_events['DIN_diff'])
fig.show()

In [ ]:
fig=px.scatter(x=et_events['time'],y=et_events['index'].diff())
fig.show()

In [ ]:
fig=px.line(x=et_raw_df['time'],y=et_raw_df['DIN'])
fig.show()

In [ ]:
scalings=dict(eyegaze=1e5)
eeg_et_raw.plot(events=eeg_events,duration=4,scalings=scalings)
#eeg_et_raw.plot(scalings=scalings)

In [ ]:
bids_path = mne_bids.BIDSPath(subject=subject_id, task=task, root=bids_root)
mne_bids.write_raw_bids(eeg_raw, bids_path, events=eeg_events, event_id=eeg_event_dict, overwrite=True, allow_preload=True, format='EDF')

In [ ]:
#plot the event latencies
fig = px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.show()

In [ ]:
#show the stimulus DIN event inter-trial intervals... this should be replicated in the eye-tracking recording..
fig = px.line(y=eeg_iti)
fig.show()